In [ ]:
# 使用具体的数值做label，小数点后6位，最后回归拟合的也是msr等值-离散程度，不是准确率。

In [ ]:
import pandas as pd
from dateutil import parser

# 定义文件路径
rawdata_path = "/data1/dxw_data/llm/redbook_final/script_next/rawdata_20%.csv"

# 读取CSV文件
rawdata = pd.read_csv(rawdata_path)

# 将日期列转换为标准格式
def parse_date(date_str):
    try:
        return parser.parse(date_str)
    except ValueError:
        return None

rawdata['post_date'] = rawdata['post_date'].apply(parse_date)

# 过滤掉无效的日期
rawdata = rawdata.dropna(subset=['post_date'])

# 输出数据格式检查
print(rawdata.head())


In [3]:
# 定义文件路径
after2monthdata_path = "/data1/dxw_data/llm/redbook_final/script_next/after2monthdata_20%.csv"

# 读取CSV文件
after2monthdata = pd.read_csv(after2monthdata_path)

# 输出数据格式检查
print(after2monthdata.head())


                  poster_id                   post_id  nums_category_img  \
0  5a5e0afb11be1017b83be966  6612057b000000001a00ce44                  9   
1  5e60e721000000000100771f  6629f7cb0000000004018e7f                  6   
2  5bd97a789333970001407645  657da21b0000000007009a23                  9   
3  617fb2900000000010007cda  64b9548c000000001201f3f4                  8   
4  5f0e7aeb000000000101d59e  64970df600000000130348ee                  4   

  post_date post_like post_collect  count   total  proportion  
0     04-07       188           55  274.0  3936.0    0.069614  
1     04-25        24           10  248.0  3936.0    0.063008  
2     12-16       667          348  230.0  2875.0    0.080000  
3     07-21       211           20   84.0  1905.0    0.044094  
4     06-24      1167          695  114.0  1662.0    0.068592  


In [4]:
import os
from PIL import Image

# 定义文件路径
image_dir = "/data1/dxw_data/llm/redbook_final/script_next/data_img_20%"

# 获取图像文件列表
image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]

# 示例读取图像
for image_file in image_files[:5]:  # 仅显示前5个示例
    image_path = os.path.join(image_dir, image_file)
    image = Image.open(image_path)
    image.show()


Error: no "view" rule for type "image/png" passed its test case
       (for more information, add "--debug=1" on the command line)
Error: no "view" rule for type "image/png" passed its test case
       (for more information, add "--debug=1" on the command line)
Error: no "view" rule for type "image/png" passed its test case
       (for more information, add "--debug=1" on the command line)
Error: no "view" rule for type "image/png" passed its test case
       (for more information, add "--debug=1" on the command line)


Error: no "view" rule for type "image/png" passed its test case
       (for more information, add "--debug=1" on the command line)


In [5]:
# 分割训练集和测试集
train_rawdata = rawdata[(rawdata['post_date'].dt.month >= 1) & (rawdata['post_date'].dt.month <= 9)]
test_rawdata = rawdata[(rawdata['post_date'].dt.month >= 10) & (rawdata['post_date'].dt.month <= 12)]

train_after2monthdata = after2monthdata[after2monthdata['post_id'].isin(train_rawdata['post_id'])]
test_after2monthdata = after2monthdata[after2monthdata['post_id'].isin(test_rawdata['post_id'])]

# 保存新的训练集和测试集CSV文件
train_rawdata_path = os.path.join("/data1/dxw_data/llm/redbook_final/script_next", "train_rawdata.csv")
test_rawdata_path = os.path.join("/data1/dxw_data/llm/redbook_final/script_next", "test_rawdata.csv")
train_rawdata.to_csv(train_rawdata_path, index=False)
test_rawdata.to_csv(test_rawdata_path, index=False)

train_after2monthdata_path = os.path.join("/data1/dxw_data/llm/redbook_final/script_next", "train_after2monthdata.csv")
test_after2monthdata_path = os.path.join("/data1/dxw_data/llm/redbook_final/script_next", "test_after2monthdata.csv")
train_after2monthdata.to_csv(train_after2monthdata_path, index=False)
test_after2monthdata.to_csv(test_after2monthdata_path, index=False)


In [28]:
import pandas as pd
from dateutil import parser
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from tqdm import tqdm
import numpy as np

# 定义文件路径
data_dir = "/data1/dxw_data/llm/redbook_final/script_next/"
rawdata_path = os.path.join(data_dir, "rawdata_20%.csv")
after2monthdata_path = os.path.join(data_dir, "after2monthdata_20%.csv")
image_dir = os.path.join(data_dir, "data_img_20%")

# 读取CSV文件
rawdata = pd.read_csv(rawdata_path)
after2monthdata = pd.read_csv(after2monthdata_path)

# 将日期列转换为标准格式
def parse_date(date_str):
    try:
        return parser.parse(date_str)
    except ValueError:
        return None

rawdata['post_date'] = rawdata['post_date'].apply(parse_date)
rawdata = rawdata.dropna(subset=['post_date'])

# 分割训练集和测试集
train_rawdata = rawdata[(rawdata['post_date'].dt.month >= 1) & (rawdata['post_date'].dt.month <= 9)]
test_rawdata = rawdata[(rawdata['post_date'].dt.month >= 10) & (rawdata['post_date'].dt.month <= 12)]

train_after2monthdata = after2monthdata[after2monthdata['post_id'].isin(train_rawdata['post_id'])]
test_after2monthdata = after2monthdata[after2monthdata['post_id'].isin(test_rawdata['post_id'])]

# 只使用1/10的数据
def get_subset_indices(data, fraction=0.05):
    data_size = len(data)
    indices = list(range(data_size))
    np.random.shuffle(indices)
    split = int(np.floor(fraction * data_size))
    return indices[:split]

train_indices = get_subset_indices(train_rawdata)
test_indices = get_subset_indices(test_rawdata)

train_rawdata = train_rawdata.iloc[train_indices]
test_rawdata = test_rawdata.iloc[test_indices]

train_after2monthdata = train_after2monthdata[train_after2monthdata['post_id'].isin(train_rawdata['post_id'])]
test_after2monthdata = test_after2monthdata[test_after2monthdata['post_id'].isin(test_rawdata['post_id'])]

# 自定义数据集类
class MultimodalDataset(Dataset):
    def __init__(self, rawdata, after2monthdata, image_dir, transform=None, max_images=1):
        self.rawdata = rawdata
        self.after2monthdata = after2monthdata
        self.image_dir = image_dir
        self.transform = transform
        self.max_images = max_images
        self.data = self._prepare_data()
    
    def _prepare_data(self):
        data = []
        for _, row in tqdm(self.rawdata.iterrows(), total=self.rawdata.shape[0], desc="Processing data"):
            poster_id = row['poster_id']
            post_id = row['post_id']
            image_files = [f for f in os.listdir(self.image_dir) if f"{poster_id}_{post_id}" in f]
            images = []
            for image_file in image_files[:self.max_images]:
                image_path = os.path.join(self.image_dir, image_file)
                image = Image.open(image_path).convert('RGB')
                if self.transform:
                    image = self.transform(image)
                images.append(image)
            # 如果图像数量少于 max_images，进行填充
            while len(images) < self.max_images:
                images.append(torch.zeros((3, 224, 224)))
            if images:
                summary = row['summary']
                label_data = self.after2monthdata[self.after2monthdata['post_id'] == post_id]['proportion']
                if not label_data.empty:
                    label = label_data.values[0]
                    data.append((summary, images, label))
        return data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        summary, images, label = self.data[idx]
        return summary, torch.stack(images), torch.tensor(label)

# 图像转换
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# 创建数据集
train_dataset = MultimodalDataset(train_rawdata, train_after2monthdata, image_dir, transform=transform)
test_dataset = MultimodalDataset(test_rawdata, test_after2monthdata, image_dir, transform=transform)

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


Processing data: 100%|██████████| 401/401 [00:23<00:00, 17.06it/s]


In [35]:
# Model Definition
import torch
import torch.nn as nn
import torch.optim as optim
from imagebind import data
from imagebind.models import imagebind_model
from imagebind.models.imagebind_model import ModalityType
from tqdm import tqdm

device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Load imagebind model
imagebind_model = imagebind_model.imagebind_huge(pretrained=True)
imagebind_model.eval()
imagebind_model.to(device)

class CrossAttentionFusion(nn.Module):
    def __init__(self, text_embedding_dim, vision_embedding_dim, common_embedding_dim, num_heads):
        super(CrossAttentionFusion, self).__init__()
        self.text_linear = nn.Linear(text_embedding_dim, common_embedding_dim)
        self.vision_linear = nn.Linear(vision_embedding_dim, common_embedding_dim)
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=common_embedding_dim, nhead=num_heads), num_layers=2)
        self.lstm = nn.LSTM(common_embedding_dim, common_embedding_dim, batch_first=True)
        self.fc = nn.Linear(common_embedding_dim, 1)
    
    def forward(self, text_embeddings, vision_embeddings):
        text_embeddings = self.text_linear(text_embeddings)
        vision_embeddings = self.vision_linear(vision_embeddings)
        # Flatten embeddings to match Transformer input requirements
        text_embeddings = text_embeddings.view(text_embeddings.size(0), -1, text_embeddings.size(-1))
        vision_embeddings = vision_embeddings.view(vision_embeddings.size(0), -1, vision_embeddings.size(-1))
        multimodal_embeddings = torch.cat((text_embeddings, vision_embeddings), dim=1)
        multimodal_embeddings = self.transformer_encoder(multimodal_embeddings)
        lstm_out, _ = self.lstm(multimodal_embeddings)
        lstm_out = lstm_out[:, -1, :]  # Take output of the last time step
        output = self.fc(lstm_out)
        return output

def get_embeddings(text_list, image_tensors):
    inputs = {
        ModalityType.TEXT: data.load_and_transform_text(text_list, device),
        ModalityType.VISION: image_tensors.to(device)
    }
    
    with torch.no_grad():
        embeddings = imagebind_model(inputs)
    return embeddings

def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    log_file = open("log.txt", "w")
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
            summaries, images, labels = batch
            images, labels = images.to(device), labels.to(device)
            text_embeddings = get_embeddings(summaries, images)
            optimizer.zero_grad()
            outputs = model(text_embeddings[ModalityType.TEXT], text_embeddings[ModalityType.VISION])
            loss = criterion(outputs, labels.unsqueeze(1))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_loader)
        log_file.write(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss}\n')
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss}')
    log_file.close()

# Initialize model, criterion, and optimizer
text_embedding_dim = 1024
vision_embedding_dim = 1024
common_embedding_dim = 768
num_heads = 8

model = CrossAttentionFusion(text_embedding_dim, vision_embedding_dim, common_embedding_dim, num_heads).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
train_model(model, train_loader, criterion, optimizer, num_epochs)

# Save the model
torch.save(model.state_dict(), os.path.join(data_dir, "multimodal_model.pth"))

print("Model training completed and saved!")

Training Epoch 1/10: 100%|██████████| 7/7 [00:02<00:00,  2.35it/s]


Epoch [1/10], Loss: 0.9850530214607716


Training Epoch 2/10: 100%|██████████| 7/7 [00:02<00:00,  2.35it/s]


Epoch [2/10], Loss: 0.017840086336946115


Training Epoch 3/10: 100%|██████████| 7/7 [00:02<00:00,  2.35it/s]


...

Epoch [8/10], Loss: 0.00022671680978549245


Training Epoch 9/10: 100%|██████████| 7/7 [00:02<00:00,  2.35it/s]


Epoch [9/10], Loss: 0.0001381988186039962


Training Epoch 10/10: 100%|██████████| 7/7 [00:02<00:00,  2.35it/s]


Epoch [10/10], Loss: 0.00011259791374738728
Model training completed and saved!


In [40]:
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

# Define function to test the model
def test_model(model, test_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        for summaries, images, labels in tqdm(test_loader, desc="Testing"):
            images, labels = images.to(device), labels.to(device)
            
            # Check for NaNs or Infinities in input data
            if torch.isnan(images).any() or torch.isinf(images).any():
                print("NaN or Infinity values found in input images")
                continue
            if torch.isnan(labels).any() or torch.isinf(labels).any():
                print("NaN or Infinity values found in input labels")
                continue

            text_embeddings = get_embeddings(summaries, images)
            outputs = model(text_embeddings[ModalityType.TEXT], text_embeddings[ModalityType.VISION])
            
            # Check for NaNs and Infinities in outputs
            if torch.isnan(outputs).any() or torch.isinf(outputs).any():
                print("NaN or Infinity values found in outputs")
                continue
            
            loss = criterion(outputs, labels.unsqueeze(1))
            if torch.isnan(loss).any() or torch.isinf(loss).any():
                print("NaN or Infinity values found in loss")
                continue
            
            test_loss += loss.item()
            
            all_predictions.extend(outputs.cpu().numpy().flatten())
            all_labels.extend(labels.cpu().numpy().flatten())
    
    avg_test_loss = test_loss / len(test_loader)
    
    # Convert lists to numpy arrays
    all_predictions = np.array(all_predictions)
    all_labels = np.array(all_labels)
    
    # Check for NaNs and Infinities in all_predictions and all_labels
    mask = ~np.isnan(all_predictions) & ~np.isinf(all_predictions) & ~np.isnan(all_labels) & ~np.isinf(all_labels)
    all_predictions = all_predictions[mask]
    all_labels = all_labels[mask]

    mse = mean_squared_error(all_labels, all_predictions)
    mae = mean_absolute_error(all_labels, all_predictions)
    r2 = r2_score(all_labels, all_predictions)
    
    print(f'Test Loss: {avg_test_loss}')
    print(f'Mean Squared Error: {mse}')
    print(f'Mean Absolute Error: {mae}')
    print(f'R2 Score: {r2}')
    
    return avg_test_loss, mse, mae, r2

# Initialize model, criterion
model = CrossAttentionFusion(text_embedding_dim, vision_embedding_dim, common_embedding_dim, num_heads).to(device)
model.load_state_dict(torch.load(os.path.join(data_dir, "multimodal_model.pth")))
criterion = nn.MSELoss()

# Test the model
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
avg_test_loss, mse, mae, r2 = test_model(model, test_loader, criterion)

print("Model testing completed!")


Testing:  15%|█▌        | 2/13 [00:00<00:05,  2.11it/s]

NaN or Infinity values found in input labels


Testing:  38%|███▊      | 5/13 [00:01<00:02,  2.67it/s]

NaN or Infinity values found in input labels


Testing:  69%|██████▉   | 9/13 [00:03<00:01,  2.58it/s]

NaN or Infinity values found in input labels


Testing: 100%|██████████| 13/13 [00:04<00:00,  2.82it/s]

Test Loss: 0.15285163418375253
Mean Squared Error: 0.19900107149744883
Mean Absolute Error: 0.4453668563594581
R2 Score: -305.49946717329817
Model testing completed!
